In [1]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST

from avalanche.evaluation.metrics import forgetting_metrics, \
accuracy_metrics, loss_metrics, timing_metrics, cpu_usage_metrics, \
confusion_matrix_metrics, disk_usage_metrics, bwt_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger, WandBLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training import Naive,EWC
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, Resize
import numpy as np
import random
import wandb
import timm
import torch
import avalanche

In [2]:
timm.models.vit_base_patch16_224(pretrained=True, num_classes= 100)

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU()
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (ls2): Identity()
     

In [3]:
seed = 2023
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)
torch.cuda.manual_seed(seed)


In [4]:

train_transform = Compose([
    Resize(224),
    ToTensor(),
    
])

test_transform = Compose([
    Resize(224),
    ToTensor(),
])


benchmark = avalanche.benchmarks.SplitCIFAR100(n_experiences=10, return_task_id=False, seed=seed, 
                       train_transform=train_transform,eval_transform = test_transform
                      )

Files already downloaded and verified
Files already downloaded and verified


In [5]:
benchmark.original_classes_in_exp

[{5, 9, 23, 37, 39, 48, 58, 62, 86, 96},
 {7, 18, 19, 28, 33, 41, 67, 69, 78, 99},
 {17, 29, 36, 44, 47, 66, 74, 80, 83, 88},
 {6, 20, 21, 27, 43, 51, 81, 82, 85, 90},
 {1, 4, 14, 63, 65, 73, 84, 91, 93, 95},
 {2, 10, 12, 32, 53, 54, 59, 75, 89, 92},
 {31, 34, 35, 40, 42, 45, 55, 64, 97, 98},
 {0, 8, 24, 38, 49, 52, 68, 71, 79, 87},
 {15, 16, 22, 25, 30, 50, 56, 57, 61, 94},
 {3, 11, 13, 26, 46, 60, 70, 72, 76, 77}]

In [6]:
benchmark.get_reproducibility_data()

{'class_ids_from_zero_from_first_exp': False,
 'class_ids_from_zero_in_each_exp': False,
 'class_mapping': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99],
 'classes_order': [39,
  48,
  62,
  58,
  37,
  9,
  5,
  86,
  96,
  23,
  41,
  69,
  18,
  7,
  67,
  78,
  99,
  28,
  19,
  33,
  17,
  36,
  29,
  80,
  88,
  44,
  66,
  47,
  83,
  74,
  51,
  81,
  90,
  20,
  82,
  27,
  85,
  6,
  43,
  21,
  95,
  14,
  84,
  1,
  73,
  91,
  4,
  93,
  63

In [7]:
model = timm.models.vit_base_patch16_224_in21k(pretrained=True, num_classes=benchmark.n_classes)
model = torch.nn.DataParallel(model)

In [8]:
model

DataParallel(
  (module): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU()
          (drop1): Dropout(p=0.0, inplace=False)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (dro

In [9]:
# loggers 
loggers = []

# log to Tensorboard
loggers.append(TensorboardLogger())

# log to text file
loggers.append(TextLogger(open('log.txt', 'a')))

# print to stdout
loggers.append(InteractiveLogger())

# W&B logger - comment this if you don't have a W&B account
# loggers.append(WandBLogger(project_name="avalanche", run_name="ewc-SplitCifar100-Vit"))

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
#     timing_metrics(epoch=True, epoch_running=True),
#     cpu_usage_metrics(experience=True),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    confusion_matrix_metrics(num_classes=benchmark.n_classes, save_image=True,
                             stream=True),
#     disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=loggers
)


2023-02-02 12:55:45.902324: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-02 12:55:46.077241: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-02 12:55:46.110237: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-02 12:55:46.788308: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [10]:
# # CREATE THE STRATEGY INSTANCE (EWC)
# cl_strategy = EWC(
#     model,
#     optimizer= torch.optim.Adam(model.parameters(), lr=0.001),
#     criterion=CrossEntropyLoss(),
#     ewc_lambda=0.1,
#     keep_importance_data=True,
#     train_mb_size=50, train_epochs=20,
#     eval_mb_size=50,device='cuda',
#     evaluator=eval_plugin)
# CREATE THE STRATEGY INSTANCE (EWC)
cl_strategy = Naive(
    model,
    optimizer= torch.optim.Adam(model.parameters(), lr=0.001),
    criterion=CrossEntropyLoss(),
    train_mb_size=50,
    train_epochs=20,
    eval_mb_size=50,
    device='cuda',
    evaluator=eval_plugin)


In [11]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start training on experience ", experience.current_experience)

    cl_strategy.train(experience)
    print("End training on experience", experience.current_experience)
    print("Computing accuracy on the test set")
    results.append(cl_strategy.eval(benchmark.test_stream[:]))
        
        
# for experience in benchmark.train_stream:
#     # train returns a dictionary which contains all the metric values
#     res = cl_strategy.train(experience)
#     print('Training completed')

#     print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
#     results.append(cl_strategy.eval(benchmark.test_stream))
    


Starting experiment...
Start training on experience  0
-- >> Start of training phase << --
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:48<00:00,  1.68s/it]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.6449
	Loss_MB/train_phase/train_stream/Task000 = 2.1283
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.1526
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.2600
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:31<00:00,  1.51s/it]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.8793
	Loss_MB/train_phase/train_stream/Task000 = 1.8242
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3216
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.4000
100%|██████████████

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [03:55<00:00,  2.35s/it]
Epoch 18 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.1577
	Loss_MB/train_phase/train_stream/Task000 = 1.2866
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5996
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.6200
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [03:57<00:00,  2.37s/it]
Epoch 19 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.2498
	Loss_MB/train_phase/train_stream/Task000 = 1.5167
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5556
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.5800
-- >> End of training phase << --
End training on experience 0
Computing accuracy on the test set
-- >> Star

KeyboardInterrupt: 

In [19]:
import wandb
wandb.finish()

ExperienceBWT/eval_phase/test_stream/Task000/Exp000,▁▁▁▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp001,▁▁▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp002,▁▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp003,▁
ExperienceForgetting/eval_phase/test_stream/Task000/Exp000,▁▁▁▁
ExperienceForgetting/eval_phase/test_stream/Task000/Exp001,▁▁▁
ExperienceForgetting/eval_phase/test_stream/Task000/Exp002,▁▁
ExperienceForgetting/eval_phase/test_stream/Task000/Exp003,▁
Loss_Epoch/train_phase/train_stream/Task000,█▆▅▅▃▃▂▂▅▃▃▃▂▂▂▂▅▃▂▂▂▂▂▄▂▂▁▁▁▁▁▄▃▂▂▂▂▁▁▂
Loss_Exp/eval_phase/test_stream/Task000/Exp000,▁▇▇██
Loss_Exp/eval_phase/test_stream/Task000/Exp001,█▁▆▇▇
